In [188]:
#libraries
import cv2
import numpy as np
from scipy.spatial.distance import euclidean

In [7]:
#read video file frame by frame
def read_video(video,skip_frames):
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    while(vid.isOpened()):
        # Capture frame-by-frame
        ret, frame = vid.read() # if ret is false, frame has no content
        # resize the video to a different resolution
        if ret:
            frame=cv2.resize(frame,(640,360))
        # skip every "skip_frame"
        if vid_length%skip_frames==0:
            frames.append(frame) #add the individual frames to a list
            vid_length+=1 #increase the vid_length counter
        if ret:
            # Display the resulting frame
#             cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    vid.release()
    cv2.destroyAllWindows()
    return frames

In [8]:
frames=read_video('IMG_2525.MOV',2)
len(frames)

1

Dominant Color Extractor

In [179]:
def dominant3(image,rgb_to_color):
    #bins dict for histograms
    bins={}
    rgb_list=[]
    
    #fill the bins dict 
    for rgb in rgb_to_color:
        bins[rgb_to_color[rgb]]=0
    #map the values of the dict to a list
    for rgb in rgb_to_color:
        rgb_list.append(rgb)
    #flatten the image to 1d 
    img = image.reshape((image.shape[0] * image.shape[1], 3))     
    
    #distance between colors and img
    for pixel in img:
        bin_aux=[]
        #get the euclidean distance between the colors and the current pixel
        for rgb in rgb_list:
            bin_aux.append(euclidean(pixel,rgb))
        # get the index of the color in rgb_list,which has the smallest distance
        min_pos = np.argmin(bin_aux)
        bins[rgb_to_color[rgb_list[min_pos]]]+=1
        
#         if euclidean(pixel,rgb_list[0])<(euclidean(pixel,rgb_list[1])):
#             if (euclidean(pixel,rgb_list[0])<euclidean(pixel,rgb_list[2])):
#                 color_key=rgb_to_color[rgb_list[0]]
#                 bins[color_key]=0
#                 bins[color_key]+=1
#             elif(euclidean(pixel,rgb_list[2])<euclidean(pixel,rgb_list[1])):
#                 color_key=rgb_to_color[rgb_list[2]]
#                 bins[color_key]=0
                
#                 bins[color_key]+=1
#             else:
#                 color_key=rgb_to_color[rgb_list[1]]
#                 bins[color_key]=0
                
#                 bins[color_key]+=1
    return bins

In [189]:
#read the file with colors and there rgb-values into a dictionary
rgb_to_color = {}
with open("colors") as f:
    for line in f:
        #split lines at "::
        color, rgb = line.strip().split(':')
        #strip the rgb-string of the parenthesis, split it up a the commas,
        #cast them to int and put them into a tuples
        rgb_value=tuple(map(int,
                            (rgb.strip('(').strip(')').split(','))))
        rgb_to_color[rgb_value] = color
len(rgb_to_color)
#purple is median purple
#skin is caucasian 

45

In [ ]:
bins=dominant3(frames[0],rgb_to_color)
print(len(bins))
print(bins)
print(np.shape(frames[0]))

(45, 3)
(230400, 3)


3. histogram